In [1]:
import os
from tqdm import tqdm as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset

In [2]:
torch.cuda.init()
device = torch.device('cuda:1')
print(torch.cuda.get_device_name(device))

GeForce GTX TITAN X


In [3]:
base_path = '/ssd2/chetanp'
batch_size = 2 ** 15

In [4]:
dataset = []
for i in range(2):
    path = os.path.join(base_path,'brown_e_base'+str(i+4)+'.pt')
    data = torch.load(path)
    dataset.append(TensorDataset(data))
dataset = ConcatDataset(dataset)
data_loader = DataLoader(dataset,batch_size = batch_size,num_workers = 14)

In [5]:
criterion = nn.CosineSimilarity()
loss_function = lambda x,y: (1-criterion(x,y)).mean()
val_losses = []
max_losses = []

In [6]:
for i in range (7):
    
    class AutoEncoder(nn.Module):

        def __init__(self):
            super(AutoEncoder, self).__init__() 

            #encode
            self.e1 = nn.Linear(9216,2048)
            self.e2 = nn.Linear(2048,2 ** (5 + i))

            #decode
            self.d1 = nn.Linear(2 ** (5 + i),2048)
            self.d2 = nn.Linear(2048,9216)

        def forward(self, x):
            encode = self.e2(F.relu(self.e1(x)))

            return self.d2(F.relu(self.d1(encode)))
        
    model = nn.DataParallel(AutoEncoder(),device_ids =[1,2]).to(device)
    path = os.path.join(base_path,f'brown_e_base_compressor_{i}.pth')
    model.load_state_dict(torch.load(path))
    model.eval()
    
    running_loss = 0
    count = 0
    max_loss = 0
    for data in tqdm(data_loader):
        ## Get Data
        sample = data[0].to(device)

        ## Pass forward
        output = model(sample)
        loss = loss_function(output,sample)

        ## Update Running Loss
        max_loss = max(loss.max().detach().item(), max_loss)
        running_loss += float(loss)
        count += 1

    print(f'Test Set Loss {i}: ', str(running_loss/count))
    val_losses.append(running_loss/count)
    print(f'Max Loss {i}: ', max_loss)
    max_losses.append(max_loss)

100%|██████████| 9/9 [00:14<00:00,  1.18s/it]


Test Set Loss 0:  0.2008543246322208
Max Loss 0:  0.2069765031337738


100%|██████████| 9/9 [00:11<00:00,  1.00s/it]


Test Set Loss 1:  0.17372238636016846
Max Loss 1:  0.1795302927494049


100%|██████████| 9/9 [00:11<00:00,  1.01it/s]


...

Test Set Loss 4:  0.0927959614329868
Max Loss 4:  0.09557731449604034


100%|██████████| 9/9 [00:12<00:00,  1.02s/it]


Test Set Loss 5:  0.0550112020638254
Max Loss 5:  0.056545063853263855


100%|██████████| 9/9 [00:12<00:00,  1.03s/it]

Test Set Loss 6:  0.04316572182708316
Max Loss 6:  0.04431982710957527


In [7]:
import csv

path = os.path.join(base_path, 'val_loss_layerwise.csv')

with open(path, 'w') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(val_losses)
    wr.writerow(max_losses)